In [35]:
#Install requirement package

!pip install requests
!pip install pandas
!pip install io
!pip install collections
!pip install re

ERROR: Could not find a version that satisfies the requirement io (from versions: none)
ERROR: No matching distribution found for io
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [67]:
#Import package

import requests
import pandas as pd
import csv
import io
from collections import defaultdict
import re
import pyodbc

In [301]:
#Set variable for PRTG

#Destination server
serverurl="https://192.168.1.187"
outputcsv="/api/table.csv?output=csvtable&count=50000"
checkdevice="&content=devices&columns=objid,host,type,device,group,location,active,probe,grpdev,notifiesx,intervalx,accessrights,dependency,probegroupdevice,status,message,priority,upsens,downsens,totalsens,downacksens,partialdownsenswarnsens,pausedsens,unusualsens,undefinedsens,totalsens"
checksensor="&content=sensors&columns=parentid,device,group,sensor,status,message,lastvalue,priority"
linktodevice="/device.htm?id="

#Credential
username=""
passhash=""

#Destination export and column label
exportname="PRTG-Device-Status-Report"
columnlabel=['Store Code', 'Device Name', 'Group Name', 'Sensor', 'Status', 'Message', 'Last Value', 'Priority', 'IPv4 Address', 'Location','Link to Device']

In [69]:
#Set link getdevice for destiation server
getdevice=serverurl+outputcsv+checkdevice+'&username='+username+'&passhash='+passhash

#Download data devices
rdevice = requests.get(getdevice,verify = False)

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [174]:
#Write csv to memory with StringIO
devices_csv_strio = io.StringIO(rdevice.text)

#Read the csv data devices
devices_csv_reader = csv.reader(devices_csv_strio, delimiter=',')

#Open the csv data devices to dataframe
#devices_df = pd.read_csv(devices_csv_strio)
#display(devices_df.tail(1))
#display(devices_df.info())

In [175]:
#Create dictionary for add to data sensor. ParentID as key, IP Address and Location as value 
devices_ip_dict = defaultdict(str)
for device in devices_csv_reader:
    devices_ip_dict[device[0]] = device[2],device[11]

In [703]:
#devices_ip_dict

In [303]:
#Set link getsensor for destiation server
getsensor=serverurl+outputcsv+checksensor+'&username='+username+'&passhash='+passhash

#Download data sensors
rsensor = requests.get(getsensor,verify = False)

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [680]:
#Write csv to memory with StringIO
sens_strio = io.StringIO(rsensor.text)

#Open the csv data sensor to dataframe
sens_df = pd.read_csv(sens_strio)

#Drop unused duplicate table
sens_df_cln=sens_df.drop(['Parent ID(RAW)', 'Group(RAW)', 'Device(RAW)', 'Sensor(RAW)', 'Status(RAW)', 'Message', 'Last Value(RAW)', 'Priority(RAW)'], axis = 1)

sens_df_cln = sens_df_cln.sort_values('Device')
sens_df_cln.reset_index(drop=True, inplace=True)

#Export dataframe to csv
sens_csv = sens_df_cln.to_csv(sep=',', index=False, encoding='utf-8')

#Write csv to memory with StringIO
sens_csv_strio = io.StringIO(sens_csv)

#Read the csv data sensors
sens_dict = csv.DictReader(sens_csv_strio)
display(sens_df_cln.tail(1))
display(sens_df_cln.info())

,Parent ID,Device,Group,Sensor,Status,Message(RAW),Last Value,Priority
9934,43911,vms.map.co.id,VMS,Ping 2022,Up,OK,1 msec,5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9935 entries, 0 to 9934
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Parent ID     9935 non-null   int64 
 1   Device        9935 non-null   object
 2   Group         9935 non-null   object
 3   Sensor        9935 non-null   object
 4   Status        9935 non-null   object
 5   Message(RAW)  9935 non-null   object
 6   Last Value    9637 non-null   object
 7   Priority      9935 non-null   int64 
dtypes: int64(2), object(6)
memory usage: 621.1+ KB


None

In [682]:
#Create empty list
output_list = list()

#Create Grouplist for exception give store code
grouplist=['floor','drc','csf','internet','server']

#Function for check value is number or not
def hasNumbers(inputString):
  return any(char.isdigit() for char in inputString)

for sens in sens_dict:
    #Create empty list
    device_info = list()
    
    #Add the devices information to the device info list.
    #device_info.append(sens['Parent ID'])
    grouplow   = sens['Group'].lower()
    device     = sens['Device'].replace(" ", "")
    deviceup   = str(device.upper())
    getscode   = deviceup[len(deviceup)-4:len(deviceup)]
    checkmin   =deviceup[deviceup.find('-')+1:len(deviceup)]
    checkmin1=checkmin[checkmin.find('-')+1:checkmin.find('-')+5]
    
    if '-' not in deviceup or 'basecamp' in grouplow or ' ssc' in grouplow or 'vms' in grouplow or 'monitoring' in grouplow or 'wan router' in grouplow or 'ccdo' in grouplow or 'wh ' in grouplow or 'floor' in grouplow or 'drc' in grouplow or 'csf' in grouplow or 'internet' in grouplow or 'server' in grouplow or 'core' in grouplow:
        storecode=('')
            
    elif deviceup.find('-') != 4 or checkmin.find('-') != 4:# and deviceup.find('-') ==4):
        storecode=(checkmin1)
        
    elif deviceup.find('-') ==4:# or (checkmin.find('-') == 4):
        if hasNumbers(checkmin[0:4]) == False:
            storecode=(getscode)
        else:
            storecode=(deviceup[5:9])
        
    else:
        storecode=('')
        
    if hasNumbers(storecode) == False:
        device_info.append(float("NaN"))
    else:
        device_info.append(storecode)
        
    device_info.append(sens['Device'])
    device_info.append(sens['Group'])
    device_info.append(sens['Sensor'])
    device_info.append(sens['Status'])
    device_info.append(sens['Message(RAW)'])
    device_info.append(sens['Last Value'])
    device_info.append(sens['Priority'])
    device_info.append(devices_ip_dict[sens['Parent ID']][0])
    device_info.append(devices_ip_dict[sens['Parent ID']][1])
    device_info.append(serverurl+linktodevice+sens['Parent ID'])
    
    #Add the device info list to the output list.
    output_list.append(device_info)

#Create dataframe from output list
output_df = pd.DataFrame(output_list, columns=columnlabel)

output_df = output_df.sort_values('Store Code')
output_df.reset_index(drop=True, inplace=True)

In [684]:
#Export data to Excel
output_df.to_excel(exportname + '.xlsx', index=None, header=True)

In [685]:
#Check result
display(output_df.head(1))
display(output_df.info())

,Store Code,Device Name,Group Name,Sensor,Status,Message,Last Value,Priority,IPv4 Address,Location,Link to Device
0,103.,MIKROTIK-DP17 (piggy back ke TS11 - 103.206.24...,DP17 - SENAYAN CITY,Ping 2635,Down,Request timed out (ICMP error # 11010),,3,10.46.2.18,,https://192.168.1.187/device.htm?id=37411
1,219,ADM-219,A219 - SUPERMALL PAKUWON SURABAYA,Ping 2850,Up,OK,22 msec,3,10.42.142.32,,https://192.168.1.187/device.htm?id=43589
2,2605,MR16 - 2605b,Meraki MR16,Ping 3037,Up,OK,1 msec,3,10.2.132.208,,https://192.168.1.187/device.htm?id=52399
3,2605,MR16 - 2605b,Meraki MR16,Ping 2507,Up,OK,2 msec,3,10.2.132.208,"Jalan Jendral Sudirman, Karet Tengsin, Jakarta",https://192.168.1.187/device.htm?id=32193
4,2704,MR16 - 2704,Meraki MR16,Ping 860,Up,OK,4 msec,3,10.2.136.7,"Jalan Jendral Sudirman, Karet Tengsin, Jakarta",https://192.168.1.187/device.htm?id=32197
...,...,...,...,...,...,...,...,...,...,...,...
95,BK03,RWAN - BK03 - GATEWAY,GRAND INDONESIA - BK03,(001) ether1 Traffic,Up,OK,17 kbit/s,3,10.10.3.254,,https://192.168.1.187/device.htm?id=21154
96,BK06,RWAN - BK06 - GATEWAY,CITOS - BK06,(001) ether1 Traffic,Up,OK,15 kbit/s,3,10.10.6.254,,https://192.168.1.187/device.htm?id=17268
97,BK06,RWAN - BK06 - GATEWAY,CITOS - BK06,(007) ether2 Traffic,Up,OK,81 kbit/s,3,10.10.6.254,,https://192.168.1.187/device.htm?id=17268
98,BK06,RWAN - BK06,CITOS - BK06,Ping 1794,Up,OK,1 msec,5,172.16.61.98,,https://192.168.1.187/device.htm?id=29701


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9935 entries, 0 to 9934
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Store Code      8372 non-null   object
 1   Device Name     9935 non-null   object
 2   Group Name      9935 non-null   object
 3   Sensor          9935 non-null   object
 4   Status          9935 non-null   object
 5   Message         9935 non-null   object
 6   Last Value      9935 non-null   object
 7   Priority        9935 non-null   object
 8   IPv4 Address    9935 non-null   object
 9   Location        9935 non-null   object
 10  Link to Device  9935 non-null   object
dtypes: object(11)
memory usage: 853.9+ KB


None

In [709]:
#Set variable for data stores
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=;"
            "Database=StoreAsset;"
            "Trusted_Connection=yes;")
query = ("SELECT TOP(1000) * FROM TokoAssetNew")

Put the DB Server123


In [697]:
#Drop empty store code
output_df.dropna(subset = ["Store Code"], inplace=True)

#Join data PRTG with input toko asset
join_df = pd.merge(output_df,data,how="left",left_on=["Store Code"],right_on=["Store#"])

In [704]:
#Initialize connection
cnxn = pyodbc.connect(cnxn_str)

#Execute the query and read to a dataframe in Python
data = pd.read_sql(query, cnxn)
data.head(1)

,Store#,StoreDesc,StoreAddr1,StoreAddr2,StoreAddr3,StoreProvin,StoreMall,StoreBrand,StoreSBU,StoreStatus,...,IPAdmin,IPPOS1,IPPOS2,IPPOS3,IPPOS4,IPFinger,SNFinger,Connection,StoreProfile,InsDate
0,00AD,User Domain,None,None,None,None,None,None,User Domain,None,...,None,None,None,None,None,None,None,None,None,2018-01-19


In [699]:
#Close the connection
del cnxn

In [700]:
filter_join_df = join_df[join_df['StoreSBU']=='MAA (MAP ACTIVE)']
filter_join_df = filter_join_df.sort_values('Store Code')
filter_join_df.reset_index(drop=True, inplace=True)
filter_join_df.drop(['Store#'], inplace=True, axis=1)

In [706]:
display(filter_join_df.head(1))
display(filter_join_df.info())

,Store Code,Device Name,Group Name,Sensor,Status,Message,Last Value,Priority,IPv4 Address,Location,...,IPAdmin,IPPOS1,IPPOS2,IPPOS3,IPPOS4,IPFinger,SNFinger,Connection,StoreProfile,InsDate
0,AD13,MIKROTIK-AD13,TUNJUNGAN PLAZA 6 SURABAYA - AD13,PING 274,Up,OK,21 msec,3,10.42.157.235,,...,-,-,-,-,-,10.100.64.38,2801100000000,WAN - Telkom VPN IP,DIRECT,2018-07-20


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 35 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Store Code      171 non-null    object
 1   Device Name     171 non-null    object
 2   Group Name      171 non-null    object
 3   Sensor          171 non-null    object
 4   Status          171 non-null    object
 5   Message         171 non-null    object
 6   Last Value      171 non-null    object
 7   Priority        171 non-null    object
 8   IPv4 Address    171 non-null    object
 9   Location        171 non-null    object
 10  Link to Device  171 non-null    object
 11  StoreDesc       171 non-null    object
 12  StoreAddr1      171 non-null    object
 13  StoreAddr2      171 non-null    object
 14  StoreAddr3      171 non-null    object
 15  StoreProvin     171 non-null    object
 16  StoreMall       171 non-null    object
 17  StoreBrand      167 non-null    object
 18  StoreSBU  

None

In [702]:
#Export data to Excel
filter_join_df.to_excel(exportname + '.xlsx', index=None, header=True)